# The Local Group Galaxy Database: An Example of a Science Analysis

In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance
from astropy import constants as cnst

from matplotlib import pyplot as plt
%matplotlib inline

from astropy.visualization import quantity_support
quantity_support()

In [ ]:
import galcat

In [ ]:
lgaldb = galcat.Database(directory='data/')
lgals = lgaldb.query_table()
lgals

In [ ]:
ax = plt.axes(projection='hammer')

msk = lgals['v_mag'] > 0 #bad data  
sc = ax.scatter(lgals['coord'].ra.wrap_at(180*u.deg).rad[msk], lgals['coord'].dec.rad[msk], 
                c=lgals['v_mag'][msk])
plt.colorbar(sc)

As expected, we see the galaxies with a pile of relatively faint ones near M31, and a gap along the Galactic ZOA.  Now lets try making "science plots" a la McConnachie's paper:

In [ ]:
# translate the distance modulus into a true "distance"
lgals['distance'] = Distance(distmod=lgals['distance_modulus'])

msk = ~lgals['radial_velocity'].mask

plt.scatter(lgals['distance'][msk], lgals['radial_velocity'][msk])

What are those odd outliers?  Lets have a look:

In [ ]:
lgals[lgals['radial_velocity']>600]

Arg!  Alternatively masked datasets.  Easy to fix, fortunately:

In [ ]:
msk = (~lgals['radial_velocity'].mask)&(lgals['radial_velocity']<999.9)

plt.scatter(lgals['distance'][msk], lgals['radial_velocity'][msk]*u.km/u.s, alpha=.8)
plt.axvline(300*u.kpc, c='k', ls='--')
plt.axvline(400*u.kpc, c='k', ls=':')
plt.xlim(0, 3*u.Mpc)

Or more physical scaling relations:

In [ ]:
mstar.value

In [ ]:
# make the wrong-but-not-horribly wrong assumption of galaxy mass-to-light ratios are 1:
mass_to_light = 1*u.solMass/u.L_sun

vabs = lgals['v_mag'] - lgals['distance'].distmod
mstar = (vabs - 4.83*u.MagUnit(1/u.Lsun)).to(u.L_sun) * mass_to_light

msk = lgals['stellar_radial_velocity_dispersion']!=0

plt.scatter(np.log10(mstar[msk]/u.Msun), lgals['stellar_radial_velocity_dispersion'][msk])
plt.xlabel(f'M* [{mstar.unit.to_string("latex")}]')

What's that upper outlier?

In [ ]:
lgals[lgals['stellar_radial_velocity_dispersion']>80*u.km/u.s]

Oh, that makes sense! It's M32, which is the only "cE" in the Local Group (it's unusually compact and dense).

Maybe instead use a mass estimator like the Wolf et al. 2010 estimator:

In [ ]:
rphys = (lgals['half-light_radius']*lgals['distance']).to(u.kpc, u.dimensionless_angles())
lgals['Mwolf'] = ((3/2) * lgals['stellar_radial_velocity_dispersion']**2 * rphys/ cnst.G).to(u.solMass)

In [ ]:
msk = np.isfinite(lgals['Mwolf'])&(lgals['Mwolf']>0)

plt.scatter(mstar[msk], lgals['Mwolf'][msk])
plt.loglog()
plt.xlabel(f'M* [{mstar.unit.to_string("latex")}]')
plt.ylabel(fr'$M_{{\rm wolf}}$ [{mstar.unit.to_string("latex")}]')

Or we can try the full 3d size-dyn mass-stellar mass scaling relation:

In [ ]:
from mpl_toolkits import mplot3d
from matplotlib import animation
from IPython import display

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = plt.axes(projection='3d')

msk = np.isfinite(lgals['Mwolf'])&(lgals['Mwolf']>0)

ax.scatter3D(np.log10(mstar[msk]/u.solMass), 
             np.log10(lgals['Mwolf'][msk]/u.solMass), 
             np.log10(rphys[msk]/u.pc))

ax.set_xlabel(fr'$\log(M_*/{u.solMass.to_string("latex_inline")[1:-1]})$', fontsize=18)
ax.set_ylabel(fr'$\log(M_{{\rm wolf}}/{u.solMass.to_string("latex_inline")[1:-1]})$', fontsize=18)
ax.set_zlabel(fr'$\log(r_{{\rm h}}/{u.solMass.to_string("latex_inline")[1:-1]})$', fontsize=18)

In [ ]:
def update_anim(frac):
    ax.azim = frac*360
              
anim = animation.FuncAnimation(fig, update_anim, np.arange(240)/240, interval=1000/30)
display.HTML(anim.to_html5_video())

# Debug 

In [ ]:
from importlib import reload

import galcat
from galcat import core
reload(core)
reload(galcat)

lgaldb = galcat.Database(directory='data/')
lgals = lgaldb.query_table()